In [ ]:
import numpy as np
from scipy.stats import norm
def tauchen(n, mu, rho, sigma):
    m = 1 / np.sqrt(1 - rho**2)
    state_space = np.linspace(mu - m*sigma, mu +m*sigma, n)
    d = (state_space[n-1] - state_space[0]) / (n-1)

    transition_matrix = np.zeros((n,n))
        for j in range(n):
    transition_matrix[0,j] = norm.cdf((state_space[0] - rho*state_space[0] + d/2)/sigma) -

In [ ]:
# setting prices
r=0.04 # interest rate
w=1 # wage